In [ ]:
!pip install transformers
!pip install evaluate
!pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

# Datasetnya SQuAD katanya ambil stanford ada banyak
link dataset https://stanfordnlp.github.io/coqa/

In [ ]:
!wget https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json
!wget https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json

--2026-01-01 18:50:41--  https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json [following]
--2026-01-01 18:50:41--  https://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49001836 (47M) [application/json]
Saving to: ‘coqa-train-v1.0.json’

coqa-train-v1.0.jso 100%[===================>]  46.73M  5.11MB/s    in 6.2s    

2026-01-01 18:50:48 (7.50 MB/s) - ‘coqa-train-v1.0.json’ saved [49001836/49001836]

--2026-01-01 18:50:48--  https://nlp.stanford.edu/data/co

In [ ]:
data = pd.read_json("/content/coqa-train-v1.0.json")
data.head()

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."


# Dataset di reduce karena training time sangat lama ( takut terkena compute limit dari colab )

jika tidak di cut training time ada di sekitar 4jam per epoch.

In [ ]:
# Sample 10% of the data
data = data.sample(frac=0.1, random_state=42).reset_index(drop=True)

print(f"New dataset size: {len(data)}")

New dataset size: 10865


In [ ]:
cols = ["text","question","answer"]

comp_list = []
for index, row in data.iterrows():
  for i in range(len(row["data"]["questions"])):
    temp_list = []
    temp_list.append(row["data"]["story"])
    temp_list.append(row["data"]["questions"][i]["input_text"])
    temp_list.append(row["data"]["answers"][i]["input_text"])
    comp_list.append(temp_list)

data = pd.DataFrame(comp_list, columns=cols)


In [ ]:
data

,text,question,answer
0,Years after throwing a bottle-up note into a l...,Where was it?,California.
1,"In biology, adaptation has three related meani...",in response to what?,the imposed conditions.
2,"Last Friday, I was coming home late after spen...",What store was the person going to?,pizza place
3,"Comcast Corporation, formerly registered as Co...",Do they own any studios?,yes
4,CHAPTER SEVENTEEN. \n\nTELLS OF DESPAIR AND A ...,Who had been comforting her?,Kunelik and her son
...,...,...,...
10860,An Internet service provider (ISP) is an organ...,When was the government thinking about letting...,compromise net neutrality
10861,"The region, as part of Lorraine, was part of t...",Into four sections?,no
10862,I've always felt a bit sad for weekend fathers...,Who is Henry?,a dog
10863,Bob woke up to go to his first day of school o...,Was he late?,yes


In [ ]:
TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 512   # Question Length
T_LEN = 128    # Target Length
BATCH_SIZE = 4
DEVICE = "cuda:0"
MODEL.to(DEVICE)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["text"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

Split dataset

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

# Training and finetuning

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(2):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1

    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")


Validation batches: 100%|██████████| 544/544 [05:37<00:00,  1.61it/s]


1/2 -> Train loss: 1.518860278785695	Validation loss: 1.04892765084172


Validation batches: 100%|██████████| 544/544 [05:37<00:00,  1.61it/s]

2/2 -> Train loss: 1.3363541898930944	Validation loss: 0.9151115133358604


In [ ]:
MODEL.save_pretrained("qa_model")
TOKENIZER.save_pretrained("qa_tokenizer")

('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
 'qa_tokenizer/added_tokens.json',
 'qa_tokenizer/tokenizer.json')

In [ ]:
def predict_answer(context, question, model, tokenizer, device):
    model.eval() # Set model to evaluation mode

    # Format input same as training
    input_text = f"question: {question}  context: {context}"

    # Tokenize input
    inputs = tokenizer(
        input_text,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).to(device)

    # Generate output tokens
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=50,      # Max length of the answer
            num_beams=4,        # Use beam search for better results
            early_stopping=True
        )

    # Decode tokens to string
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
import evaluate
squad_metric = evaluate.load("squad")

def evaluate_model(model, tokenizer, data_loader, device):
    model.eval()
    predictions = []
    references = []

    print("Running evaluation...")
    for batch in tqdm(data_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        # For SQuAD metric, we need a specific format
        # In this simplified version, we just look at raw string overlap
        for p, label_ids in zip(preds, batch["labels"]):
            # Convert label_ids back to text, ignoring -100
            label_ids[label_ids == -100] = tokenizer.pad_token_id
            ref = tokenizer.decode(label_ids, skip_special_tokens=True)

            predictions.append(p)
            references.append(ref)

    # Simple Accuracy check for generative text
    em = sum([1 for p, r in zip(predictions, references) if p.strip().lower() == r.strip().lower()]) / len(predictions)
    print(f"Validation Exact Match (EM): {em:.4f}")

evaluate_model(MODEL, TOKENIZER, val_loader, DEVICE)

Running evaluation...


100%|██████████| 544/544 [02:47<00:00,  3.24it/s]

Validation Exact Match (EM): 0.5435


In [ ]:
# 1. test the function
my_context = "The Golden State Warriors are an American professional basketball team based in San Francisco. They have won seven NBA championships."
my_question = "How many championships do the Warriors have?"

# 2. Call the function
prediction = predict_answer(my_context, my_question, MODEL, TOKENIZER, DEVICE)

# 3. See the result
print(f"Question: {my_question}")
print(f"Model Answer: {prediction}")

Question: How many championships do the Warriors have?
Model Answer: seven


In [ ]:
my_context = "The Bandung Club has won eighty championships"
my_question = "How many championships does The Bandung Club have?"

# 2. Call the function
prediction = predict_answer(my_context, my_question, MODEL, TOKENIZER, DEVICE)

# 3. See the result
print(f"Question: {my_question}")
print(f"Model Answer: {prediction}")

Question: How many championships does The Bandung Club have?
Model Answer: eighty
